In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import math
import cv2

Y_START,Y_END=60,135
IMG_SIZE=(200,66)
BATCH = 32




### Data Augmentation

### Data preprocessing


In [ ]:
def preprocess_image(img_bgr):
    
    # Crop the image to get the road
    cropped = img_bgr[60:136, :, :]
    
    # Convert BGR to YUV
    yuv = cv2.cvtColor(cropped, cv2.COLOR_BGR2YUV)

    # Gaussian blur
    yuv = cv2.GaussianBlur(yuv, (3, 3), 0)

    # Resize
    resized = cv2.resize(yuv, (200, 66), interpolation=cv2.INTER_AREA)

    # Normalize pixel values to range[0,1]
    processed = resized.astype(np.float32) / 255.0

    return processed

### Model 